In [27]:
## Creating shp file with new F, G, H labels to override model predictions 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import os, sys
import pickle
sys.path.append('scripts/')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio, rasterio.plot
import shapely as shp
import xarray as xr
import rioxarray as rxr
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import loadpaths
import land_cover_analysis as lca
import land_cover_visualisation as lcv
import land_cover_models as lcm

In [46]:
## Load OS data:

dir_gpkg = '/home/tplas/data/gis/os_mastermap/'

dict_gpkg_file_names = {'roads': 'trn_fts_roadtrackorpath.gpkg',
                        'buildings': 'bld_fts_buildingpart.gpkg',
                        'sites': 'lus_fts_site.gpkg',
                        'structures': 'str_fts_structure.gpkg',
                        'railways': 'trn_fts_rail.gpkg',
                        'water': 'wtr_fts_water.gpkg',
                        'land': 'lnd_fts_land.gpkg',}

dict_gdfs = {}
dict_unique_descriptions = {}

for key, value in dict_gpkg_file_names.items():
    path_layer = os.path.join(dir_gpkg, value)
    dict_gdfs[key] = gpd.read_file(path_layer)
    dict_unique_descriptions[key] = dict_gdfs[key]['description'].unique()

    ## Assert all geometry types are polygons
    set_unique_geom_types = set([type(x) for x in dict_gdfs[key]['geometry']])
    for geom_type in set_unique_geom_types:
        assert geom_type in [shp.geometry.polygon.Polygon, shp.geometry.multipolygon.MultiPolygon], f'Unexpected geometry type {geom_type} for {key}'

In [62]:
## Create mapping dict template (from OS description to LC label)

os_to_lc_mapping = {key: {} for key in dict_gdfs.keys()}  # separate dict for each OS layer
min_size_dict = {key: None for key in dict_gdfs.keys()}

## Roads:
road_exceptions = ['Cycle Way', 'Path', 'Path And Steps', 'Track']  # all too minor to be considered transport routes
assert (np.isin(road_exceptions, dict_unique_descriptions['roads'])).all(), 'Road exceptions not found in OS data'
for descr in dict_unique_descriptions['roads']:
    if descr not in road_exceptions:
        os_to_lc_mapping['roads'][descr] = 'H1b'

# ## Buildings:
# for descr in dict_unique_descriptions['buildings']:
#     os_to_lc_mapping['buildings'][descr] = 'H1a'
## Better to use site layer .. ? [Private Residential Site]

## Sites:
other_isol_dev_list = ['Fish Farm']
skip_dev_list = ['Amenity And Open Space Site', 'Mine', 'Outdoor Activity Centre']
assert (np.isin(skip_dev_list, dict_unique_descriptions['sites'])).all(), 'Skip development exceptions not found in OS data'
assert (np.isin(other_isol_dev_list, dict_unique_descriptions['sites'])).all(), 'Other isolated development exceptions not found in OS data'
for descr in dict_unique_descriptions['sites']:
    if descr == 'Farm Site':
        os_to_lc_mapping['sites'][descr] = 'H3a'
    elif descr in other_isol_dev_list:
        os_to_lc_mapping['sites'][descr] = 'H3b' 
    elif descr == 'Quarry':
        os_to_lc_mapping['sites'][descr] = 'H2a'  # some false positives in here! 
    elif descr in ['Rail Freight Transport' 'Rail Maintenance Site' 'Railway Station']:
        os_to_lc_mapping['sites'][descr] = 'H1b'
    elif descr in skip_dev_list:
        continue
    else:
        os_to_lc_mapping['sites'][descr] = 'H1a'

## Structures 
## Think we can ignore these; mostly covered by sites for urban staff anyway. otherwise lots of tiny structures that are not relevant.

## Railways:
for descr in dict_unique_descriptions['railways']:
    os_to_lc_mapping['railways'][descr] = 'H1b'

## Water:
water_exceptions = ['Collects', 'Drain', 'Leat', 'Mill Leat', 'Open Tank Reservoir', 'Open Water Tank', 
                    'Overflow', 'Paddling Pool', 'Reed Bed For Waste Water', 'Settling Pond', 'Spreads',
                    'Spring', 'Swimming Pool', 'Watercourse', 'Waterfall']
assert (np.isin(water_exceptions, dict_unique_descriptions['water'])).all(), 'Water exceptions not found in OS data'
for descr in dict_unique_descriptions['water']:
    os_to_lc_mapping['water'][descr] = 'F2'
## Maybe add minimum area threshold for water? 
min_size_dict['water'] = 100

## Land:
rock_list = ['Boulders Or Rock']
urban_land_list = ['Made Surface']
assert (np.isin(urban_land_list, dict_unique_descriptions['land'])).all(), 'Urban land exceptions not found in OS data'
assert (np.isin(rock_list, dict_unique_descriptions['land'])).all(), 'Rock exceptions not found in OS data'
for descr in dict_unique_descriptions['land']:
    if descr in rock_list:
        os_to_lc_mapping['land'][descr] = 'G2'
    elif descr in urban_land_list:
        os_to_lc_mapping['land'][descr] = 'H3b'




In [83]:
total_gdf = None 
tmp_dict = {} 
## Loop through different layres & append to total_gdf
for it, (key, gdf_layer) in enumerate(dict_gdfs.items()):
    inds_use = gdf_layer['description'].isin(os_to_lc_mapping[key].keys())  # only use features with a mapping
    print(f'Using {np.sum(inds_use)}/{len(gdf_layer)} features from {key} layer')
    tmp_layer = gdf_layer.loc[inds_use, :].copy()
    
    if min_size_dict[key] is not None:  # apply area threshold if specified
        tmp_layer = tmp_layer.loc[tmp_layer['geometry'].area > min_size_dict[key], :]
        print(f'Using {len(tmp_layer)} features from {key} layer after area thresholding')

    tmp_layer['lc_label'] = tmp_layer['description'].map(os_to_lc_mapping[key]) # add LC label
    tmp_dict[key] = tmp_layer.copy()
    if it == 0: 
        total_gdf = tmp_layer.copy()
    else:
        total_gdf = total_gdf.append(tmp_layer, ignore_index=True)
    

Using 24745/34716 features from roads layer
Using 0/56423 features from buildings layer
Using 19704/19729 features from sites layer
Using 0/3089 features from structures layer
Using 328/328 features from railways layer
Using 6943/6943 features from water layer
Using 3682 features from water layer after area thresholding
Using 19968/180798 features from land layer


In [86]:
## Remove any overlapping features

## Use overlay method? (maybe in original loop when appending dfs)
gpd.overlay(tmp_dict['roads'], tmp_dict['sites'], how='difference')

,osid,toid,versiondate,versionavailablefromdate,versionavailabletodate,firstdigitalcapturedate,changetype,geometry_area,geometry_evidencedate,geometry_updatedate,...,oslanduse_evidencedate,oslanduse_updatedate,oslanduse_source,istidal,associatedstructure,isobscured,physicallevel,capturespecification,geometry,lc_label
1,fdde02da-5f8d-48cb-999b-bd129c3464b3,osgb5000005199518763,2022-08-26,2022-08-27T00:00:00,None,2017-02-06,New,17.295285,2016-07-19,2017-02-03,...,2016-07-19,2017-02-03,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((419773.650 385145.320, 419769.652 38...",H1b
2,fddb3070-339d-43e4-8e22-afc93a8b3fd4,osgb1000000105686060,2022-08-26,2022-08-27T00:00:00,None,2001-04-25,New,815.726168,2014-07-24,2015-05-01,...,2006-08-30,2006-08-30,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((420651.670 375372.090, 420645.660 37...",H1b
5,fde17502-e3c9-4f57-92e5-4691c49dbb66,osgb1000000102780612,2022-08-26,2022-08-27T00:00:00,None,2000-09-26,New,2883.853510,2014-03-12,2014-09-04,...,2010-03-08,2010-03-08,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((402907.740 390423.020, 402908.167 39...",H1b
6,fde92f2d-110e-418f-b43a-db22e065417a,osgb1000000108090839,2022-08-26,2022-08-27T00:00:00,None,2000-09-21,New,35.283650,2005-08-23,2005-08-23,...,2005-08-23,2005-08-23,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((405125.420 354566.600, 405125.750 35...",H1b
8,fde8fca8-3d3f-43b2-bc34-e426ff089368,osgb1000000102789754,2022-08-26,2022-08-27T00:00:00,None,2003-01-09,New,38.761000,2004-06-08,2004-06-08,...,2004-06-08,2004-06-08,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((417381.610 381006.260, 417382.810 38...",H1b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34710,6e153079-fb87-4169-bac1-bf9976f97f52,osgb1000000108174749,2022-12-15,2022-12-16T00:00:00,None,2000-09-21,Modified Geometry And Attributes,1506.901800,2022-12-14,2022-12-14,...,2005-09-01,2005-09-01,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((409884.300 354353.510, 409885.320 35...",H1b
34711,ba4336e7-9c49-47eb-af3b-9742808c1edb,osgb1000000108377679,2022-12-14,2022-12-15T00:00:00,None,2005-10-13,Modified Geometry And Attributes,1267.570060,2022-12-08,2022-12-08,...,2014-05-17,2014-12-08,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((406201.000 365280.790, 406222.380 36...",H1b
34712,6e8790f8-9629-445f-82c2-960988d0b9fa,osgb1000002492157859,2022-12-16,2022-12-17T00:00:00,None,2004-12-08,Modified Geometry And Attributes,378.460034,2022-12-15,2022-12-15,...,2011-01-12,2011-01-12,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((423009.720 375508.500, 423009.930 37...",H1b
34713,373aae56-714d-42d7-8050-3e6b2f1fa97c,osgb5000005143286867,2022-12-22,2022-12-23T00:00:00,None,2015-02-27,Modified Geometry And Attributes,494.259532,2022-12-12,2022-12-12,...,2014-05-17,2015-02-13,Ordnance Survey,False,None,False,Surface Level,Rural,"POLYGON ((396118.590 364006.140, 396123.390 36...",H1b


In [55]:
dict_gdfs['sites'].head()

,osid,toid,versiondate,versionavailablefromdate,versionavailabletodate,changetype,geometry_area,geometry_evidencedate,geometry_updatedate,geometry_source,...,name1_updatedate,name1_source,name2_text,name2_language,name2_evidencedate,name2_updatedate,name2_source,primaryuprn,extentdefinition,geometry
0,0007103d-031a-43ec-a943-13fef3e4c0be,None,2022-10-03,2022-10-04T00:00:00,None,New,152.955750,2021-06-22,2021-06-22,Ordnance Survey,...,None,None,None,None,None,None,None,NaN,Auto-Defined Complete,"MULTIPOLYGON (((415287.430 382969.390, 415288...."
1,000126ba-e5a8-44d4-8d59-f0f9b48df6a9,None,2022-10-03,2022-10-04T00:00:00,None,New,81.996546,2021-06-22,2021-06-22,Ordnance Survey,...,2011-06-25,Ordnance Survey,None,None,None,None,None,NaN,Auto-Defined Complete,"MULTIPOLYGON (((415130.290 375979.630, 415127...."
2,0004d726-904d-4b66-814c-bed2671e6059,None,2022-10-03,2022-10-04T00:00:00,None,New,2386.002895,2022-02-14,2022-02-14,Ordnance Survey,...,2011-06-25,Ordnance Survey,None,None,None,None,None,NaN,Manually Defined Complete,"MULTIPOLYGON (((412303.540 385472.850, 412300...."
3,00062644-c052-474e-82d6-70f3071b54c2,None,2022-10-03,2022-10-04T00:00:00,None,New,1077.241186,2021-12-13,2021-12-13,Ordnance Survey,...,2011-06-25,Ordnance Survey,None,None,None,None,None,NaN,Manually Defined Complete,"MULTIPOLYGON (((411271.250 361298.880, 411274...."
4,00067186-9934-4e64-89ac-80d6c41b3ab2,None,2022-10-03,2022-10-04T00:00:00,None,New,256.549544,2021-06-22,2021-06-22,Ordnance Survey,...,2018-01-24,Ordnance Survey,None,None,None,None,None,NaN,Auto-Defined Complete,"MULTIPOLYGON (((409440.960 366681.720, 409438...."


In [53]:
for key, descr_list in dict_unique_descriptions.items():
    if key == 'land':
        print(key, len(descr_list))
        print(np.sort(descr_list))
        print('\n\n')

land 73
['Arable Or Grazing Land' 'Bare Earth Or Grass' 'Boulders Or Rock'
 'Boulders Or Rock And Heath Or Rough Grassland'
 'Boulders Or Rock And Heath Or Rough Grassland And Marsh'
 'Boulders Or Rock And Heath Or Rough Grassland And Scattered Coniferous Trees'
 'Boulders Or Rock And Heath Or Rough Grassland And Scattered Non-Coniferous Trees'
 'Boulders Or Rock And Heath Or Rough Grassland Or Marsh'
 'Boulders Or Rock And Scattered Mixed Trees'
 'Boulders Or Rock And Scattered Non-Coniferous Trees' 'Coniferous Trees'
 'Coniferous Trees And Scattered Boulders Or Scattered Rock'
 'Coniferous Trees And Scattered Boulders Or Scattered Rock And Scrub'
 'Coniferous Trees And Scrub' 'Construction Site' 'Gallops' 'Games Court'
 'Heath' 'Heath Or Rough Grassland' 'Heath Or Rough Grassland And Marsh'
 'Heath Or Rough Grassland And Marsh And Scattered Coniferous Trees'
 'Heath Or Rough Grassland And Marsh And Scattered Non-Coniferous Trees'
 'Heath Or Rough Grassland And Marsh And Scrub'
 'Heat